In [ ]:
df = pd.read_json("../database-unclean/all/train_period_data.jsonlist", orient='list', lines=True)


In [ ]:
def grab_top_comment(post_id):
   submission = reddit.submission(id=post_id)
   submission.submission_type = 'best'
   submission.comments.replace_more(limit=0)
   comments = list(submission.comments)[0].replies.list()
   non_op_responses = []
   for c in comments:
      #print(c.author, " ", submission.author)
      if c.author is not None:
         if c.author.name == submission.author.name:
            #print("boop")
            non_op_responses.append("OP: " + c.body)
      else:
         non_op_responses.append(c.body)

   return non_op_responses

In [9]:
comments = grab_top_comment("2r3lm3")

Voted_Quimby   IIIBlackhartIII
IIIBlackhartIII   IIIBlackhartIII
boop
Outofmany   IIIBlackhartIII
loercase   IIIBlackhartIII
stevegcook   IIIBlackhartIII
bunker_man   IIIBlackhartIII
garnteller   IIIBlackhartIII
THE_LAST_HIPPO   IIIBlackhartIII
None   IIIBlackhartIII
garnteller   IIIBlackhartIII
garnteller   IIIBlackhartIII
None   IIIBlackhartIII
Voted_Quimby   IIIBlackhartIII
loercase   IIIBlackhartIII
Voted_Quimby   IIIBlackhartIII
almightySapling   IIIBlackhartIII
Outofmany   IIIBlackhartIII
loercase   IIIBlackhartIII
loercase   IIIBlackhartIII
NJFiend   IIIBlackhartIII
stevegcook   IIIBlackhartIII
garnteller   IIIBlackhartIII
garnteller   IIIBlackhartIII
sailinator   IIIBlackhartIII
garnteller   IIIBlackhartIII
kingpatzer   IIIBlackhartIII
pantaloonsofJUSTICE   IIIBlackhartIII
None   IIIBlackhartIII
solomon34   IIIBlackhartIII
_Roark   IIIBlackhartIII
Voted_Quimby   IIIBlackhartIII
Voted_Quimby   IIIBlackhartIII
Outofmany   IIIBlackhartIII
loercase   IIIBlackhartIII
loercase   IIIB

In [10]:
thingy = " ".join(comments)

In [11]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("pszemraj/long-t5-tglobal-base-16384-book-summary")

model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/long-t5-tglobal-base-16384-book-summary")

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

C:\Users\yokon\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yokon\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
import torch
from transformers import pipeline

summarizer = pipeline(
   "summarization",
   "pszemraj/long-t5-tglobal-base-16384-book-summary",
   device=0 if torch.cuda.is_available() else -1,
)

result = summarizer(thingy)

C:\Users\yokon\anaconda3\lib\site-packages\transformers\modeling_utils.py:722: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
print(result[0]['summary_text'])